In [1]:
from memote.support.helpers import find_met_in_model, find_compartment_id_in_model
import cobra
from cobra.exceptions import Infeasible
from os.path import join

data_dir = "/Users/lizrad/Dev/iVnat"
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))

In [7]:
with model:
    for rxn in model.boundary:
        rxn.bounds = 0, 0
        
    for rxn in model.boundary:
        print(rxn.name)
        rxn.bounds = -10, 10
        sol = model.optimize()
        print(sol)

DNA_replication_c0
<Solution 0.000 at 0x1116e0a20>
RNA_transcription_c0
<Solution 0.000 at 0x1116e09b0>
Protein_biosynthesis_c0
<Solution 0.000 at 0x1116e08d0>
EX_H_plus__e0
<Solution 0.000 at 0x1116e0f60>
EX_Adenosine_e0
<Solution 0.000 at 0x1116e0a20>
EX_Fumarate_e0
<Solution 0.000 at 0x1116e0978>
EX_Nitrite_e0
<Solution 0.000 at 0x1116e08d0>
EX_Ornithine_e0
<Solution 0.000 at 0x1116e09b0>
EX_4_5_dihydroxy_2_3_pentanedione_c0
<Solution 0.000 at 0x1116e0a20>
EX_Succinate_e0
<Solution 0.000 at 0x1116e0f60>
EX_L_methionine_R_oxide_e0
<Solution 0.000 at 0x1116e08d0>
EX_Pb_e0
<Solution 0.000 at 0x1116e0978>
EX_L_alanylglycine_e0
<Solution 0.000 at 0x1116e0c88>
EX_Deoxyuridine_e0
<Solution 0.000 at 0x1116e0fd0>
EX_ala_L_Thr_L_e0
<Solution 0.000 at 0x1116e0e10>
EX_PPi_e0
<Solution 0.000 at 0x1116e0978>
EX_ala_L_glu_L_e0
<Solution 0.000 at 0x1116e0940>
EX_Salicin_e0
<Solution 0.000 at 0x1116e0fd0>
EX_L_Proline_e0
<Solution 0.000 at 0x1116e0a20>
EX_Na_plus__e0
<Solution 0.000 at 0x1116e0978>


In [34]:
blocked_precursors = []
with model:
    for rxn in model.boundary:
        print(rxn.name)
        rxn.bounds = -10, 0
        sol = model.optimize()
        print(sol)


    for precursor in [model.metabolites.ACP_c0, model.metabolites.ADENOSYLCOBALAMIN_c0, model.metabolites.MNXM7630_c0]:
        dm_rxn = model.add_boundary(precursor, type="demand")
        model.objective = dm_rxn
        try:
            print("demand reaction", dm_rxn)
            solution = model.optimize()


            print(solution)
            LOGGER.debug(
                "%s: demand flux is '%g' and solver status is '%s'",
                str(precursor), solution.objective_value, solution.status)

            if solution.objective_value <= 0.0:
                blocked_precursors.append(precursor)
                print("obj value 0 or negative:", blocked_precursors)
        except Infeasible:
            blocked_precursors.append(precursor)


    

DNA_replication_c0
<Solution 0.000 at 0x1153b3400>
RNA_transcription_c0
<Solution 0.000 at 0x1153a3c88>
Protein_biosynthesis_c0
<Solution 0.000 at 0x1153b39b0>
EX_H_plus__e0
<Solution 0.000 at 0x1153b3cf8>
EX_Adenosine_e0
<Solution 0.000 at 0x1153b3400>
EX_Fumarate_e0
<Solution 0.000 at 0x1153a3c88>
EX_Nitrite_e0
<Solution 0.000 at 0x1153b39b0>
EX_Ornithine_e0
<Solution 0.000 at 0x1153b3cf8>
EX_4_5_dihydroxy_2_3_pentanedione_c0
<Solution 0.000 at 0x1153b3400>
EX_Succinate_e0
<Solution 0.000 at 0x1153b3cf8>
EX_L_methionine_R_oxide_e0
<Solution 0.000 at 0x1153b3a20>
EX_Pb_e0
<Solution 0.000 at 0x1153b3cf8>
EX_L_alanylglycine_e0
<Solution 0.000 at 0x1153b31d0>
EX_Deoxyuridine_e0
<Solution 0.000 at 0x1153b3cf8>
EX_ala_L_Thr_L_e0
<Solution 0.000 at 0x1153b3160>
EX_PPi_e0
<Solution 0.000 at 0x1153b3cf8>
EX_ala_L_glu_L_e0
<Solution 0.000 at 0x1153b3278>
EX_Salicin_e0
<Solution 0.000 at 0x1153b3cf8>
EX_L_Proline_e0
<Solution 0.000 at 0x1153b3400>
EX_Na_plus__e0
<Solution 0.000 at 0x1153b3cf8>


In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
#from cobra.medium import minimal_medium
#from cobra.medium import minimal_medium
from IPython.core.interactiveshell import InteractiveShell
import itertools
import re
import memote

InteractiveShell.ast_node_interactivity = "all"

data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("/Users/lizrad/Documents/Vibrio_folder/iVnat_gapfill_glu.xml")
model

files found: 
iVnat.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x010daba7f0
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


## Memote biomass and consistency functions

In [ ]:
#from memote.support.biomass import find_blocked_biomass_precursors, sum_biomass_weight, find_biomass_precursors, find_direct_metabolites, gam_in_biomass, detect_false_positive_direct_metabolites, bundle_biomass_components, essential_precursors_not_in_biomass

In [ ]:
#from memote.support.consistency import check_stoichiometric_consistency
#from memote.support.helpers import find_compartment_id_in_model

### Finding biomass precursors

In [ ]:
bio_precursor=find_biomass_precursors(model, model.reactions.bio1)

print("Number of found biomass precursors:" , len(bio_precursor))
bio_precursor

In [ ]:
len(model.reactions.bio1.metabolites)
for precursor in model.reactions.bio1.metabolites:
    if precursor not in bio_precursor:
        print(precursor)


### Missing essential precursors

In [ ]:
missing_essential=essential_precursors_not_in_biomass(model, model.reactions.bio1)
missing_essential

len(missing_essential)
for_real=[]
for bio in model.reactions.bio1.metabolites:
    if bio.annotation["metanetx.chemical"] in missing_essential:
        for_real.append(bio.annotation["metanetx.chemical"])
        print(bio.annotation["metanetx.chemical"])
        bio.compartment
    
        
len(for_real)


main_comp = find_compartment_id_in_model(model, 'c')
main_comp


#memote looks for ids not in the annotation field, my IDs contain c0 

In [ ]:

list_missing_met=[]        
for met in model.metabolites:
    for mis in missing_essential_precursors:
        if mis in met.annotation.values():
           
            list_missing_met.append(met.id)
            print(met.id, mis)
            
len(list_missing_met)       
#37 vs. 52 missing metabolite--> duplication in e0/c0



In [ ]:
both=set()      #because of duplicate ids c0 e0  
for met in model.metabolites:
    for mis in missing_essential_precursors:
        if mis in met.annotation.values():
            both.add(mis)
            
len(both)            
print(both)            
  
#no missing essential precursors

### Blocked biomass precursors

In [ ]:
#blocked=find_blocked_biomass_precursors(model, model.reactions.bio1)

blocked_bio=find_blocked_biomass_precursors(model.reactions.bio1, model)
blocked_bio

### Sum biomass weight

In [ ]:
with model:
    
    for met in model.metabolites:
        
        met.formula=re.sub("(([R])\d+)", "", met.formula)
        met.formula=re.sub("([R])" , "", met.formula)
        met.formula=re.sub("(MISSING)" , "", met.formula)
        
    #sum_bio=sum_biomass_weight(model.reactions.bio1)
    #sum_bio
       
            

In [ ]:
sum_bio=sum_biomass_weight(model.reactions.bio1)
sum_bio
       

### Growth associated maintenance

In [ ]:
#growth-associated maintenance
#True if the biomass reaction includes ATP and H2O as reactants and ADP,Pi and H as products, False otherwise.
gam=gam_in_biomass(model, model.reactions.bio1)
gam
model.reactions.bio1.reaction
#What does it check(id?)?

model.metabolites.MNXM2214_c0
model.metabolites.MNXM5258_c0
model.metabolites.MNXM90619_c0



### False positive direct metabolites

### Bundle biomass components

In [ ]:
bundle=bundle_biomass_components(model, model.reactions.bio1)
bundle

### Find direct metabolites

In [ ]:

with model: 
    #for met in model.metabolites:
     #   if "(" in met.formula:
      #      print(met.formula, met.id)
            
  
    model.metabolites.MNXM8231_c0.formula="C21H33N5O9P2"
   
    model.metabolites.get_by_id("DNA-6-O-Methyl-Guanines_c0").formula="C10H16N2NO2O6S"
    model.metabolites.MNXM8231_c0.formula
    model.metabolites.get_by_id("DNA-6-O-Methyl-Guanines_c0").formula
    direct_met=find_direct_metabolites(model, model.reactions.bio1, tolerance=1e-06)
    
    direct_met
   
   #     met.formula=re.sub("(([R])\d+)", "", met.formula)
   #     met.formula=re.sub("(([R]))" , "", met.formula)
   #     met.formula=re.sub("(MISSING)" , "", met.formula)
        
    sum_bio=sum_biomass_weight(model.reactions.bio1)
    sum_bio
    

### Stoichiometric consistency

In [ ]:
stoich=check_stoichiometric_consistency(model)
stoich

### Memote repair

from memote import test_model

In [ ]:
#_, result = test_model(model, results=True)

In [ ]:
#result

### Reset the exchange fluxes to zero

Change the exchange fluxes to zero. Then add the ones that are necessary for minimal media. (What if there is no media that we want? How does the media work ? Where does it show? 
Create a dictionary where there is a readable format for the medium. Is there an annotation field for medium? 

In [ ]:
#model.reactions.EX_cpd00009_e0.bounds = (0,1000)

    
with model:
    
    for reac in model.reactions:
        if reac.id.startswith("EX") or  ["MNXR137716_c0", 'MNXR137717_c0', 'MNXR137715_c0']:
            reac.bounds=(0,0)
        
    model.medium 
    model.reactions.EX_cpd00012_e0.bounds = (-1000,1000) #ppi
    model.reactions.EX_cpd00009_e0.bounds = (-1000,1000) #pi
    model.reactions.EX_cpd00209_e0.bounds = (-1000,1000) #NO3
    model.reactions.EX_cpd00075_e0.bounds = (-1000,1000) #NO2
    model.reactions.EX_cpd00013_e0.bounds = (-1000,1000) #NH4
    model.reactions.EX_cpd00048_e0.bounds = (-1000,1000) #SO4
    model.reactions.EX_cpd00027_e0.bounds = (-1000,1000) #D-glucose
    model.reactions.EX_cpd00971_e0.bounds = (-1000,1000) #Na+
    model.reactions.EX_cpd00076_e0.bounds = (-1000,1000) #Sucrose
    model.reactions.EX_cpd00080_e0.bounds = (-1000,1000) #D-Glycerol 1-phosphate
    model.reactions.EX_cpd00001_e0.bounds = (-1000,1000) #H2O
    model.reactions.EX_cpd00011_e0.bounds = (-1000,1000) #CO2
    model.reactions.EX_cpd00067_e0.bounds = (-1000,1000) #H+
    model.reactions.EX_cpd00063_e0.bounds = (-1000,1000) #Ca2+
    model.reactions.EX_cpd00099_e0.bounds = (-1000,1000) #Cl-
    model.reactions.EX_cpd00205_e0.bounds = (-1000,1000) #K+
    model.reactions.EX_cpd00254_e0.bounds = (-1000,1000) #Mg2+
    model.reactions.EX_cpd00030_e0.bounds = (-1000,1000) #Mn2+
    model.reactions.EX_cpd00007_e0.bounds = (-1000,1000) #O2
    model.reactions.MNXR137716_c0.bounds = (-1000,1000) #
    model.reactions.MNXR137715_c0.bounds = (-1000,1000) #
    model.reactions.MNXR137717_c0.bounds = (-1000,1000) #
    model.reactions.EX_cpd00058_e0.bounds = (-1000,1000) #Cu2+
    model.reactions.EX_cpd10515_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd10516_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00034_e0.bounds = (-1000,1000) #Fe2+
    
    model.reactions.EX_cpd00035_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00039_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00041_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd11580_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00023_e0.bounds = (-1000,1000) #Fe2
    model.slim_optimize()
    

    model.medium
    #model.optimize()
    
    model.reactions.MNXR137715_c0

### Existing media and their description to dict

In [ ]:
chem_xref = read_csv("/Users/lizrad/Documents/Vibrio_folder/chem_xref.tsv" , sep='\t' , low_memory=False)
chem_xref
#499091 530559

df_seed = chem_xref.loc[499088:530556,'XREF':'Description']
pivoted_seed = df_seed.pivot_table(index='XREF',columns='XREF_ID',values='Description',aggfunc = lambda x: x)

#pivoted_seedd=pivoted_seed.drop(['metanetx'])
pivoted_seed
chem_xref_MNX_seed_dict = pivoted_seed.to_dict('index')
chem_xref_MNX_seed_dict2 = chem_xref_MNX_seed_dict['seed']
print(chem_xref_MNX_seed_dict2)


In [ ]:
#MNXR137716_c0','MNXR137717_c0','MNXR137715_c0' dont have seed IDs
medium_dict={}
for med in model.medium:
    if med.startswith("MNXR"):
        continue
    pre, seed, compartment=med.split("_")
    print(seed)
    for k in chem_xref_MNX_seed_dict2.keys():
        if seed in k: 
            print (chem_xref_MNX_seed_dict2[k])
            medium_dict[pre+"_"+seed+"_"+compartment]=chem_xref_MNX_seed_dict2[k]
            
print(medium_dict)

In [ ]:
#check every combinations of mediums :(  
#for L in range(0, len(model.medium)+1):
#    for subset in itertools.combinations(model.medium, L):
#        print(subset)

From KBase download medium in tsv. Try LB(Lysogeny Broth) and minimal nms(nitrate minimal salts medium). With Edit Media Application in KBase one can modify the existing medium) 

Result: No growth (neither).
Check for blocked precursors: both blocked with 76 metabolites. 


In [ ]:
media_nms = read_csv("/Users/lizrad/Documents/Vibrio_folder/LB.tsv" , sep='\t' , low_memory=False)
#media_nms = read_csv("/Users/lizrad/Documents/Vibrio_folder/minimal_nms.tsv" , sep='\t' , low_memory=False)
media_nms

comp_list=media_nms["compounds"].tolist()

with model:
    for met in model.metabolites: 
        if met.annotation["seed.compound"] in comp_list: 
            print(met.id, met.annotation["seed.compound"])

        
    #Adding the direct metabolites as exchange reactions
         
    for reac in model.reactions:
        if reac.id.startswith("EX"):
            
            reac.bounds=(0,0)
            reac.id
            pre, seed, compart =reac.id.split("_")
            if seed in comp_list: 
                reac.bounds=(-1000,1000)
            for i in direct_met:
                meta, compart =i.id.split("_")
                if meta in seed:
                    reac.bounds
                    reac.bounds=(-1000,1000)
                    reac.bounds
    for d in direct_met:
        model.add_boundary(model.metabolites.get_by_id(d.id), type="exchange", reaction_id=None, lb=-1000.0, ub=1000.0)
    #
    model.medium
    len(model.medium)
    model.optimize().objective_value
    model.summary()
    
   # blocked_precursors=find_blocked_biomass_precursors(model.reactions.bio1, model)
   # blocked_precursors
   # len(blocked_precursors)
#model.reactions.get_by_id['EX_cpd00067_e0'].bounds

    

In [ ]:
direct_list=[]
with model:
    
    for d in direct_met:
        metanet,suff =d.id.split("_")
        direct_list.append(d.id)
        
direct_list



In [ ]:
direct_met_reaction_dict={}
for reac in model.reactions:
    for rmet in reac.metabolites:
    
        if  rmet in direct_met:
            #direct_met_reaction_dict[direct_met].append(rmet)
            print(rmet, "is in the reaction of ", reac)
            
            


Setting the direct precursors to medium, those that are already in the medium are found, the other direct metabolites cannot be found among the mediums. 

In [ ]:
 
with model:
    
    for d in direct_list:
        pre1, suff1 =d.split("_")
        
        
        #create a new metabolite in the e0 compartment
        mcopy=model.metabolites.get_by_id("K+_c0").copy()
        mcopy.compartment = 'e0'
        mcopy.id = pre1+"_e0"
        mcopy.formula=""
        mcopy.name=""
        model.add_metabolites(mcopy)
        
        mcopy
        #create a new reaction between the c0 and e0 compartment
        
        rcopy=model.reactions.get_by_id("TRANS-RXN-143_c0").copy()
        rcopy.id="TRANS_"+d
        for r in rcopy.metabolites:
            r.remove_from_model
        #model.reactions.get_by_id(rcopy).subtract_metabolites()   
        rcopy.name=""
        rcopy.gene_reaction_rule=""
        pre, suff=mcopy.id.split("_")
        rcopy.add_metabolites({
        model.metabolites.get_by_id(pre+"_c0"): -1.0, model.metabolites.get_by_id(mcopy.id): 1.0 })
        rcopy.reaction
        
model.reactions.get_by_id("TRANS-RXN-143_c0")   


In [ ]:
from memote.support.helpers import find_transport_reactions

In [ ]:
for rxn in model.metabolites.MNXM5129_c0.reactions:
    rxn
    if rxn in find_transport_reactions(model):
        print(rxn.id)

In [ ]:
for met in model.reactions.MNXR136333_c0.metabolites:
    print(met.compartment, met.formula)
model.reactions.MNXR136333_c0.copy

In [ ]:
for met in model.reactions.MNXR136261_c0.metabolites:
    print(met, met.compartment, met.formula, met.annotation)
model.reactions.MNXR136261_c0.reaction
model.reactions.MNXR136261_c0.annotation

In [ ]:
for met in model.reactions.MNXR136333_c0.metabolites:
    print(met, met.compartment, met.formula)
model.reactions.MNXR136333_c0.reaction

In [ ]:
len(find_transport_reactions(model))


In [ ]:

with model: 
   # add_ex = model.add_boundary(model.metabolites.MNXM6175_c0, type="exchange")
    #add_ex.id
    
    for d in direct_met:
        if d not in model.medium:
            metd=d.id
            model.metabolites.get_by_id(metd)
            model.add_boundary(model.metabolites.get_by_id(d.id), type="exchange", reaction_id="EX"+"_"+metd, lb=-1000, ub=1000)
        #model.reactions.add(add_ex)
        #model.reactions.EX
    for reac in model.reactions:
        
        if reac.id.startswith("EX"):
            print(reac.id)
    
    model.medium
    model.optimize()
    model.exchanges
    for reac in model.reactions:
        B=is_boundary_type(reac, "exchange", "c0")
    
        if B :
            reac.id
    
        

In [ ]:
from memote.support.helpers import find_biomass_reaction
with model:
    model.reactions.bio1.annotation['sbo'] = 'SBO:0000629'
    #model.reactions.bio1.annotation['SBO'] = 'SBO:0000629'
    model.reactions.bio1.annotation
    #model.reactions.bio1.id="biomass"

    biomass_reac=find_biomass_reaction(model)
    biomass_reac

#write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")
    

In [ ]:
#what is the id for the selected substrates from seed id
substrate=["cpd00076","cpd00027", "cpd00080","cpd00179", "cpd00082", "cpd00036", "cpd00224", "cpd00314", "cpd00020","cpd00029","cpd00154",
           "cpd00208","cpd00138","cpd00158","cpd00363","cpd00116","cpd00047"]

for met in model.metabolites:
    if met.annotation["seed.compound"] in substrate and met.compartment=="c0":
        print(met.id, met.annotation["seed.compound"] )

        
#does the subsrate have exchange reaction        

ex_substrate=[]        
for exchange in medium_dict.keys():
    ex, seed, suff=exchange.split("_")
    if  seed in substrate:
        print(exchange, medium_dict[exchange])
        ex_substrate.append(exchange)




### Transport reaction bug

In [ ]:
from __future__ import absolute_import

import logging
import re
from collections import defaultdict
from operator import itemgetter


import numpy as np
import pandas as pd
from six import iteritems, itervalues
from sympy import expand
from importlib_resources import open_text
from cobra.exceptions import Infeasible
from cobra.medium import find_boundary_types
from pylru import lrudecorator

import memote.utils as utils
import memote.support.data
import memote.utils as utils
import memote.support.data
from memote.support.helpers import find_biomass_reaction

In [ ]:
import memote
import cobra

In [ ]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs
model2 = Model('example_model')

#def abc_pump_annotations(base):
   # """Provide a model with an ABC transport reaction."""
atp = cobra.Metabolite("atp_c", compartment="c")
adp = cobra.Metabolite("adp_c", compartment="c")
h = cobra.Metabolite("h_c", formula="H",compartment="c")
pi = cobra.Metabolite("pi_c", compartment="c")
h2o = cobra.Metabolite("h2o_c", formula="H2O", compartment="c")
aso_c = cobra.Metabolite("aso3_c", compartment="c")
aso_e = cobra.Metabolite("aso3_e", compartment="e")

atp.annotation["biocyc"] = ["META:ATP", "META:CPD0-1634"]
adp.annotation["biocyc"] = ["META:ADP", "META:CPD0-1651"]
h.annotation["biocyc"] = "META:PROTON"
pi.annotation["biocyc"] = ["META:CPD-16459", "META:CPD-9010"]
h2o.annotation["biocyc"] = ["META:CPD-15815", "META:HYDROXYL-GROUP"]
aso_c.annotation["biocyc"] = ["META:CPD0-2040", "META:CPD-763"]
aso_e.annotation["biocyc"] = ["META:CPD0-2040", "META:CPD-763"]

pump = cobra.Reaction("PUMP")
pump.add_metabolites({aso_c: -1, atp: -1, h2o: -1,
                          adp: 1, h: 1, pi: 1, aso_e: 1})
model2.add_reactions([pump])
####################
MNXM11392_c0 = cobra.Metabolite("MNXM11392_c0", formula="C40H78O13P2", compartment="c" )
MNXM5129_c0 = cobra.Metabolite("MNXM5129_c0", formula="C40H78O10P", compartment="c" )
CPD_16459_c0 = cobra.Metabolite("CPD_16459_c0", formula="HO4P", compartment="c" )

#MNXM11392_c0 = cobra.Metabolite("MNXM11392_c0",  compartment="c" )
#MNXM5129_c0 = cobra.Metabolite("MNXM5129_c0",  compartment="c" )
#CPD_16459_c0=cobra.Metabolite("MNXM5129_c0",  compartment="c" )
h2o.annotation['bigg.metabolite']=['h2o', 'oh1']
h2o.annotation["chebi"]=['CHEBI:15377']
h2o.annotation['inchi']=['InChI=1S/H2O/h1H2']
MNXM11392_c0.annotation={'inchi': 'MISSING', 'inchikey': 'MISSING', 'metanetx.chemical': 'MNXM11392', 'seed.compound': 'cpd15716', 'smiles': 'MISSING'}
CPD_16459_c0.annotation={'bigg.metabolite': 'pi', 'biocyc': ['CPD-16459', 'CPD-9010', 'PHOSPHATE-GROUP', 'Pi'], 'chebi': 'CHEBI:43474', 'inchi': 'InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4)/p-2', 'inchikey': 'NBIIXXVUZAFLBC-UHFFFAOYSA-L', 'kegg.compound': ['C00009', 'C13558'], 'metanetx.chemical': 'MNXM9', 'seed.compound': 'cpd00009', 'smiles': 'OP([O-])([O-])=O'}
MNXM5129_c0.annotation={'inchi': 'MISSING', 'inchikey': 'MISSING', 'metanetx.chemical': 'MNXM5129', 'seed.compound': 'cpd15722', 'smiles': 'MISSING'}



MNXR136261_c0=cobra.Reaction("MNXR136261_c0")
MNXR136261_c0.add_metabolites({ MNXM11392_c0: -1, h2o: -1,
                           h: 1, CPD_16459_c0: 1, MNXM5129_c0: 1})

model2.add_reactions([MNXR136261_c0])
#'CPD-15815_c0 + MNXM11392_c0 --> CPD-16459_c0 + MNXM5129_c0 + PROTON_c0'
####################


GLYCEROL_c0 = cobra.Metabolite("GLYCEROL_c0", formula="C3H8O3", compartment="c" )
MNXM6297_c0 = cobra.Metabolite("MNXM6297_c0", formula="C77H148O17P2", compartment="c" )


MNXR136333_c0=cobra.Reaction("MNXR136333_c0", lower_bound=-1000)
MNXR136333_c0.add_metabolites({ MNXM5129_c0: -2, GLYCEROL_c0: 1,
                           MNXM6297_c0: 1})

model2.add_reactions([MNXR136333_c0])




model2
    #return base
print('%i reaction' % len(model2.reactions))
print('%i metabolites' % len(model2.metabolites))
print('%i genes' % len(model2.genes))
pump.reaction
MNXR136261_c0.reaction
MNXR136333_c0.reaction
MNXM5129_c0.annotation
    


In [3]:
def find_biomass_reaction(model):
    """
    Return a list of the biomass reaction(s) of the model.
    This function identifies possible biomass reactions using two steps:
    1. Return reactions that include the SBO annotation "SBO:0000629" for
    biomass.
    If no reactions can be identifies this way:
    2. Look for the ``buzzwords`` "biomass", "growth" and "bof" in reaction IDs.
    3. Look for metabolite IDs or names that contain the ``buzzword`` "biomass"
    and obtain the set of reactions they are involved in.
    4. Remove boundary reactions from this set.
    5. Return the union of reactions that match the buzzwords and of the
    reactions that metabolites are involved in that match the buzzword.
    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    Returns
    -------
    list
        Identified biomass reactions.
    """
    sbo_matches = set([rxn for rxn in model.reactions if
                       rxn.annotation is not None and
                       'SBO' in rxn.annotation and
                       rxn.annotation['SBO'] == 'SBO:0000629'])

    if len(sbo_matches) > 0:
        return list(sbo_matches)

    buzzwords = ['biomass', 'growth', 'bof']

    buzzword_matches = set([rxn for rxn in model.reactions if any(
        string in rxn.id.lower() for string in buzzwords)])

    biomass_met = []
    for met in model.metabolites:
        if met.id.lower().startswith('biomass') or met.name.lower().startswith(
            'biomass'
        ):
            biomass_met.append(met)
    if biomass_met == 1:
        biomass_met_matches = set(
            biomass_met.reactions
        ) - set(model.boundary)
    else:
        biomass_met_matches = set()

    return list(buzzword_matches | biomass_met_matches)


In [4]:
def is_transport_reaction_formulae(rxn):
    """
    Return boolean if a reaction is a transport reaction (from formulae).
    Parameters
    ----------
    rxn: cobra.Reaction
        The metabolic reaction under investigation.
    """
    # Collecting criteria to classify transporters by.
    rxn_reactants = set([met.formula for met in rxn.reactants])
    rxn_products = set([met.formula for met in rxn.products])
    # Looking for formulas that stay the same on both side of the reaction.
    transported_mets = \
        [formula for formula in rxn_reactants if formula in rxn_products]
    # Collect information on the elemental differences between
    # compartments in the reaction.
    delta_dicts = find_transported_elements(rxn)
    non_zero_array = [v for (k, v) in iteritems(delta_dicts) if v != 0]
    # Excluding reactions such as oxidoreductases where no net
    # transport of Hydrogen is occurring, but rather just an exchange of
    # electrons or charges effecting a change in protonation.
    if set(transported_mets) != set('H') and list(
        delta_dicts.keys()
    ) == ['H']:
        pass
    # All other reactions for which the amount of transported elements is
    # not zero, which are not part of the model's exchange nor
    # biomass reactions, are defined as transport reactions.
    # This includes reactions where the transported metabolite reacts with
    # a carrier molecule.
    elif sum(non_zero_array):
        return True

In [5]:
def find_transported_elements(rxn):
    """
    Return a dictionary showing the amount of transported elements of a rxn.
    Collects the elements for each metabolite participating in a reaction,
    multiplies the amount by the metabolite's stoichiometry in the reaction and
    bins the result according to the compartment that metabolite is in. This
    produces a dictionary of dictionaries such as this
    ``{'p': {'C': -1, 'H': -4}, c: {'C': 1, 'H': 4}}`` which shows the
    transported entities. This dictionary is then simplified to only include
    the non-zero elements of one single compartment i.e. showing the precise
    elements that are transported.
    Parameters
    ----------
    rxn : cobra.Reaction
        Any cobra.Reaction containing metabolites.
    """
    element_dist = defaultdict()
    # Collecting elements for each metabolite.
    for met in rxn.metabolites:
        if met.compartment not in element_dist:
            # Multiplication by the metabolite stoichiometry.
            element_dist[met.compartment] = \
                {k: v * rxn.metabolites[met]
                 for (k, v) in iteritems(met.elements)}
        else:
            x = {k: v * rxn.metabolites[met] for (k, v) in
                 iteritems(met.elements)}
            y = element_dist[met.compartment]
            element_dist[met.compartment] = \
                {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}
    delta_dict = defaultdict()
    # Simplification of the resulting dictionary of dictionaries.
    for elements in itervalues(element_dist):
        delta_dict.update(elements)
    # Only non-zero values get included in the returned delta-dict.
    delta_dict = {k: abs(v) for (k, v) in iteritems(delta_dict) if v != 0}
    return delta_dict


In [ ]:
def is_transport_reaction_annotations(rxn):
    """
    Return boolean if a reaction is a transport reaction (from annotations).
    Parameters
    ----------
    rxn: cobra.Reaction
        The metabolic reaction under investigation.
    """
    reactants = set([(k, tuple(v)) for met in rxn.reactants
                     for k, v in iteritems(met.annotation)
                     if met.id is not "H" and k is not None and v is not None])
    #print("reactants", reactants)
    products = set([(k, tuple(v)) for met in rxn.products
                    for k, v in iteritems(met.annotation)
                    if met.id is not "H" and k is not None and v is not None])
    #print("products", products)
    # Find intersection between reactant annotations and
    # product annotations to find common metabolites between them,
    # satisfying the requirements for a transport reaction. Reactions such
    # as those involving oxidoreductases (where no net transport of
    # Hydrogen is occurring, but rather just an exchange of electrons or
    # charges effecting a change in protonation) are excluded.
    transported_mets = reactants & products
    #print("transported_mets", transported_mets, len(transported_mets))
    
    if len(transported_mets) > 0:
        return True


In [ ]:
def find_transport_reactions(model):
    """
    Return a list of all transport reactions.
    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    Notes
    -----
    A transport reaction is defined as follows:
    1. It contains metabolites from at least 2 compartments and
    2. at least 1 metabolite undergoes no chemical reaction, i.e.,
    the formula and/or annotation stays the same on both sides of the equation.
    A notable exception is transport via PTS, which also contains the following
    restriction:
    3. The transported metabolite(s) are transported into a compartment through
    the exchange of a phosphate group.
    An example of transport via PTS would be
    pep(c) + glucose(e) -> glucose-6-phosphate(c) + pyr(c)
    Reactions similar to transport via PTS (referred to as "modified transport
    reactions") follow a similar pattern:
    A(x) + B-R(y) -> A-R(y) + B(y)
    Such modified transport reactions can be detected, but only when a formula
    field exists for all metabolites in a particular reaction. If this is not
    the case, transport reactions are identified through annotations, which
    cannot detect modified transport reactions.
    """
    transport_reactions = []
    transport_rxn_candidates = set(model.reactions) - set(model.boundary) \
        - set(find_biomass_reaction(model))
    #print("transport_rxn_candidates",transport_rxn_candidates)
    # Add all labeled transport reactions
    sbo_matches = set([rxn for rxn in transport_rxn_candidates if
                       rxn.annotation is not None and
                       'SBO' in rxn.annotation and
                       rxn.annotation['SBO'] in TRANSPORT_RXN_SBO_TERMS])
    if len(sbo_matches) > 0:
        transport_reactions += list(sbo_matches)
    # Find unlabeled transport reactions via formula or annotation checks
    for rxn in transport_rxn_candidates:
        # Check if metabolites have formula field
        rxn_mets = set([met.formula for met in rxn.metabolites])
        if (None not in rxn_mets) and (len(rxn_mets) != 0):
            if is_transport_reaction_formulae(rxn):
                transport_reactions.append(rxn)
        elif is_transport_reaction_annotations(rxn): #
            transport_reactions.append(rxn)
       # print("rxn_mets",rxn_mets)
    return set(transport_reactions)

    

In [ ]:
for rxn in model.metabolites.MNXM5129_c0.reactions:
    
    if rxn in find_transport_reactions(model):
        print(rxn.id)

In [ ]:
model2_trans=memote.support.helpers.find_transport_reactions(model2)
model2_trans
#find_direct_metabolites(model, bio1)

In [ ]:
model2_bio=find_biomass_reaction(model2)
model2_bio

In [ ]:
find_transported_elements(MNXR136261_c0)

In [ ]:
istranspann=is_transport_reaction_annotations(MNXR136261_c0)
istranspann

### Blocked precursors bug

In [ ]:
missing_essential=essential_precursors_not_in_biomass(model, model.reactions.bio1)
missing_essential


In [20]:
def find_biomass_precursors(model, reaction):
    """
    Return a list of all biomass precursors excluding ATP and H2O.
    Parameters
    ----------
    reaction : cobra.core.reaction.Reaction
        The biomass reaction of the model under investigation.
    model : cobra.Model
        The metabolic model under investigation.
    Returns
    -------
    list
        Metabolite objects that are reactants of the biomass reaction excluding
        ATP and H2O.
    """
    id_of_main_compartment =find_compartment_id_in_model(model, 'c')
    print(id_of_main_compartment)
    gam_reactants = set()
    try:
        gam_reactants.update([
            find_met_in_model(
                model, "MNXM3", id_of_main_compartment)[0]])
    except RuntimeError:
        pass
    try:
        gam_reactants.update([
            find_met_in_model(
                model, "MNXM2", id_of_main_compartment)[0]])
    except RuntimeError:
        pass

    biomass_precursors = set(reaction.reactants) - gam_reactants
    print("number of gam", gam_reactants)
    
    

    return list(biomass_precursors)

In [ ]:
met = cobra.Metabolite('test')
rxn = cobra.Reaction(id='test2')
rxn.add_metabolites({met: 1})

In [ ]:
rxn.add_metabolites({met: -1})

In [ ]:
rxn

In [18]:

import logging
LOGGER = logging.getLogger(__name__)
def find_blocked_biomass_precursors(reaction, model):
    
    LOGGER.debug("Finding blocked biomass precursors")
    precursors = find_biomass_precursors(model, reaction)
    precursors
    blocked_precursors = list()
    for precursor in precursors:
        precursor
        with model:
            dm_rxn = model.add_boundary(precursor, type="demand")
            
            model.objective = dm_rxn
            try:
                print("demand reaction", dm_rxn)
                solution = model.optimize()
                
                
                print(solution)
                LOGGER.debug(
                    "%s: demand flux is '%g' and solver status is '%s'",
                    str(precursor), solution.objective_value, solution.status)
                
                if solution.objective_value <= 0.0:
                    blocked_precursors.append(precursor)
                    print("obj value 0 or negative:", blocked_precursors)
            except Infeasible:
                blocked_precursors.append(precursor)
                
    return blocked_precursors

In [10]:
from memote.support.helpers import find_met_in_model, find_compartment_id_in_model
x=find_biomass_precursors(model, model.reactions.bio1)

len(x)
len(model.reactions.bio1.reactants)
model.metabolites.ACP_c0
model.metabolites.ADENOSYLCOBALAMIN_c0
model.metabolites.MNXM7630_c0

c0
number of gam set()


76

76

Metabolite identifier,ACP_c0
Name,ACP
Memory address,0x011043cb38
Formula,HS
Compartment,c0
In 63 reaction(s),"MNXR134926_c0, MNXR134906_c0, MNXR134950_c0, MNXR135829_c0, RXN-9531_c0, MALONYL-COA-ACP-TRANSACYL-RXN_c0, RXN-9539_c0, MNXR134901_c0, MNXR134951_c0, MNXR135825_c0, MNXR134885_c0, MNXR135826_c0, MN..."


Metabolite identifier,ADENOSYLCOBALAMIN_c0
Name,Calomide
Memory address,0x011080e550
Formula,C72H102CoN18O17P
Compartment,c0
In 2 reaction(s),"COBALAMINSYN-RXN_c0, bio1"


Metabolite identifier,MNXM7630_c0
Name,Peptidoglycan_polymer_n_subunits
Memory address,0x01108c9588
Formula,C80H125N16O42
Compartment,c0
In 2 reaction(s),"bio1, MNXR136199_c0"


In [ ]:
with model:
    rxn=model.add_boundary(model.metabolites.ACP_c0, type="exchange")
    model.objective = rxn
    solution = model.optimize()
    solution
    lin=linear_reaction_coefficients(model)
    lin
    solution.status
    model.summary()
    solution.fluxes
    solution.shadow_prices
    rxn.bounds
    model.metabolites.ACP_c0.reactions
    

In [12]:
from cobra.util.solver import linear_reaction_coefficients
blocked_bio_precursor=find_blocked_biomass_precursors(model.reactions.bio1, model)


c0
number of gam set()
demand reaction DM_CPD-12818_c0: CPD-12818_c0 --> 
<Solution 500.000 at 0x10eb409b0>
demand reaction DM_ASN_c0: ASN_c0 --> 
<Solution 1000.000 at 0x10eb405f8>
demand reaction DM_DEMETHYLMENAQUINONE_c0: DEMETHYLMENAQUINONE_c0 --> 
<Solution 125.000 at 0x10eb40da0>
demand reaction DM_UNDECAPRENYL-DIPHOSPHATE_c0: UNDECAPRENYL-DIPHOSPHATE_c0 --> 
<Solution 90.909 at 0x10eb40c50>
demand reaction DM_CTP_c0: CTP_c0 --> 
<Solution 666.667 at 0x10eb40ef0>
demand reaction DM_PRO_c0: PRO_c0 --> 
<Solution 1000.000 at 0x10eb40978>
demand reaction DM_TYR_c0: TYR_c0 --> 
<Solution 1000.000 at 0x10eb40860>
demand reaction DM_MNXM81014_c0: MNXM81014_c0 --> 
<Solution 1000.000 at 0x10eb40780>
demand reaction DM_L-ALPHA-ALANINE_c0: L-ALPHA-ALANINE_c0 --> 
<Solution 1000.000 at 0x10eb40080>
demand reaction DM_MNXM7305_c0: MNXM7305_c0 --> 
<Solution 83.333 at 0x10eb18438>
demand reaction DM_DCTP_c0: DCTP_c0 --> 
<Solution 666.667 at 0x10eb186d8>
demand reaction DM_CO-A_c0: CO-A_c0 -

In [72]:
print(model.objective)

Maximize
-1.0*bio1_reverse_b18f7 + 1.0*bio1


In [73]:
model.optimize()

,fluxes,reduced_costs
H2PTEROATESYNTH-RXN_c0,0.000000,-4.295657e-02
GLUTATHIONE-SYN-RXN_c0,0.000000,-8.591313e-02
MNXR101695_c0,0.000000,-0.000000e+00
HYPOXANPRIBOSYLTRAN-RXN_c0,0.000000,-4.295657e-02
RXN-8641_c0,0.000000,-1.734723e-18
...,...,...
EX_cpd00011_e0,1000.000000,1.073914e-02
EX_cpd11416_c0,397.348245,0.000000e+00
EX_cpd00099_e0,-1.230376,0.000000e+00
EX_cpd00063_e0,-1.230376,0.000000e+00


In [76]:
with model: 
    #bio_copy=model.reactions.bio1.copy()
    #bio_copy
    #model.add_reaction(bio_copy)
    
    model.reactions.bio1.metabolites
    model.reactions.bio1.add_metabolites({model.metabolites.ACP_c0: +0.003096466851925328, model.metabolites.ADENOSYLCOBALAMIN_c0: +0.003096466851925328, model.metabolites.MNXM7630_c0: +0.025010597710894533 })
    model.reactions.bio1.metabolites
    #for reac in model.reactions:
    #    if reac.id.startswith("EX"):
    #        model.reactions.bio1.bounds=(0,0)
    
with model:
    reac_list=[]
    
    for blo in blocked_bio_precursor:
        blo
        dm_rxn = model.add_boundary(blo, type="demand", reaction_id='test')
        model.objective = dm_rxn.id
        print(model.objective)
        reac_list.append(dm_rxn)
       # dm_rxn.bounds=(0,0)
        print ('#####'*50)
        model.optimize()
        print ('#####'*50)
        model.summary
    
    #model.reactions.bio1.metabolites
    #model.reactions.bio1.add_metabolites({model.metabolites.ACP_c0: -1, model.metabolites.ADENOSYLCOBALAMIN_c0: -1, model.metabolites.MNXM7630_c0:-1 })
    #model.add_reactions(reac_list)
    
with model:
    blocked_fx=find_blocked_biomass_precursors(model.reactions.bio1, model)
    print ('Blocked Precursors')
    blocked_fx
   
    model.optimize()
    model.summary
    
        
    model.metabolites.ACP_c0.reactions
    model.reactions.MNXR109353_c0
        

{<Metabolite 10-FORMYL-THF_c0 at 0x110499b38>: -0.00309646685192537,
 <Metabolite 5-METHYL-THF_c0 at 0x11053f630>: -0.00309646685192537,
 <Metabolite ARG_c0 at 0x1105c7780>: -0.246696822701341,
 <Metabolite ASN_c0 at 0x110533080>: -0.200830806928348,
 <Metabolite ATP_c0 at 0x11043b7f0>: -40.1101757365074,
 <Metabolite CA+2_c0 at 0x110910f60>: -0.00309646685192537,
 <Metabolite CL-_c0 at 0x1105b45c0>: -0.00309646685192537,
 <Metabolite CO+2_c0 at 0x1107c5a20>: -0.00309646685192537,
 <Metabolite CO-A_c0 at 0x110497390>: -0.00309646685192537,
 <Metabolite CPD-12818_c0 at 0x11058c048>: -0.0106480421341882,
 <Metabolite CPD-12822_c0 at 0x1104fc748>: -0.0106480421341882,
 <Metabolite CPD-15815_c0 at 0x11045be48>: -35.5403092430435,
 <Metabolite CPD-9728_c0 at 0x110583780>: -0.00309646685192537,
 <Metabolite CPD0-2271_c0 at 0x110910b70>: -0.0250105977108944,
 <Metabolite CTP_c0 at 0x110499128>: -0.0841036156544863,
 <Metabolite CU+2_c0 at 0x1104f09e8>: -0.00309646685192537,
 <Metabolite CYS_c

{<Metabolite 10-FORMYL-THF_c0 at 0x110499b38>: -0.00309646685192537,
 <Metabolite 5-METHYL-THF_c0 at 0x11053f630>: -0.00309646685192537,
 <Metabolite ARG_c0 at 0x1105c7780>: -0.246696822701341,
 <Metabolite ASN_c0 at 0x110533080>: -0.200830806928348,
 <Metabolite ATP_c0 at 0x11043b7f0>: -40.1101757365074,
 <Metabolite CA+2_c0 at 0x110910f60>: -0.00309646685192537,
 <Metabolite CL-_c0 at 0x1105b45c0>: -0.00309646685192537,
 <Metabolite CO+2_c0 at 0x1107c5a20>: -0.00309646685192537,
 <Metabolite CO-A_c0 at 0x110497390>: -0.00309646685192537,
 <Metabolite CPD-12818_c0 at 0x11058c048>: -0.0106480421341882,
 <Metabolite CPD-12822_c0 at 0x1104fc748>: -0.0106480421341882,
 <Metabolite CPD-15815_c0 at 0x11045be48>: -35.5403092430435,
 <Metabolite CPD-9728_c0 at 0x110583780>: -0.00309646685192537,
 <Metabolite CPD0-2271_c0 at 0x110910b70>: -0.0250105977108944,
 <Metabolite CTP_c0 at 0x110499128>: -0.0841036156544863,
 <Metabolite CU+2_c0 at 0x1104f09e8>: -0.00309646685192537,
 <Metabolite CYS_c

Metabolite identifier,ADENOSYLCOBALAMIN_c0
Name,Calomide
Memory address,0x011080e550
Formula,C72H102CoN18O17P
Compartment,c0
In 2 reaction(s),"COBALAMINSYN-RXN_c0, bio1"


Maximize
1.0*DM_ADENOSYLCOBALAMIN_c0 - 1.0*DM_ADENOSYLCOBALAMIN_c0_reverse_ae856
##########################################################################################################################################################################################################################################################


,fluxes,reduced_costs
H2PTEROATESYNTH-RXN_c0,0.000000,0.0
GLUTATHIONE-SYN-RXN_c0,0.000000,0.0
MNXR101695_c0,0.000000,-0.0
HYPOXANPRIBOSYLTRAN-RXN_c0,0.000000,0.0
RXN-8641_c0,0.000000,-0.0
...,...,...
EX_cpd11416_c0,397.348245,0.0
EX_cpd00099_e0,-1.230376,0.0
EX_cpd00063_e0,-1.230376,0.0
EX_cpd00030_e0,-1.230376,0.0


...

[<Metabolite ADENOSYLCOBALAMIN_c0 at 0x11080e550>,
 <Metabolite MNXM7630_c0 at 0x1108c9588>,
 <Metabolite ACP_c0 at 0x11043cb38>]

,fluxes,reduced_costs
H2PTEROATESYNTH-RXN_c0,0.000000,-4.295657e-02
GLUTATHIONE-SYN-RXN_c0,0.000000,-8.591313e-02
MNXR101695_c0,0.000000,-0.000000e+00
HYPOXANPRIBOSYLTRAN-RXN_c0,0.000000,-4.295657e-02
RXN-8641_c0,0.000000,-1.734723e-18
...,...,...
EX_cpd00011_e0,1000.000000,1.073914e-02
EX_cpd11416_c0,397.348245,0.000000e+00
EX_cpd00099_e0,-1.230376,0.000000e+00
EX_cpd00063_e0,-1.230376,0.000000e+00


<bound method Model.summary of <Model GCF_001456255_1_rast_metabolic_model at 0x10daba7f0>>

frozenset({<Reaction 3-OXOACYL-ACP-SYNTH-BASE-RXN_c0 at 0x11132fda0>,
           <Reaction ACP-S-ACETYLTRANSFER-RXN_c0 at 0x1115665c0>,
           <Reaction MALONYL-COA-ACP-TRANSACYL-RXN_c0 at 0x1112f78d0>,
           <Reaction MNXR109301_c0 at 0x111c45160>,
           <Reaction MNXR109353_c0 at 0x1115fabe0>,
           <Reaction MNXR111734_c0 at 0x11132fac8>,
           <Reaction MNXR111735_c0 at 0x1117bf5c0>,
           <Reaction MNXR134855_c0 at 0x110ac66a0>,
           <Reaction MNXR134856_c0 at 0x110f6ee48>,
           <Reaction MNXR134860_c0 at 0x11137d4e0>,
           <Reaction MNXR134864_c0 at 0x110b7b390>,
           <Reaction MNXR134868_c0 at 0x110f521d0>,
           <Reaction MNXR134872_c0 at 0x1118eda90>,
           <Reaction MNXR134876_c0 at 0x1115e5588>,
           <Reaction MNXR134880_c0 at 0x1119dbb70>,
           <Reaction MNXR134881_c0 at 0x111850470>,
           <Reaction MNXR134885_c0 at 0x11135c160>,
           <Reaction MNXR134889_c0 at 0x110b18fd0>,
           <R

Reaction identifier,MNXR109353_c0
Name,R05146_c0
Memory address,0x01115fabe0
Stoichiometry,Dodecanoyl-ACPs_c0 + KDO2-LIPID-IVA_c0 --> ACP_c0 + MNXM1870_c0 Dodecanoyl_ACP + kdo2_lipid_iva --> ACP + Lauroyl_KDO2_lipid_IVA
GPR,PN96_RS05160 or PN96_RS12525 or PN96_RS05160_CDS_1 or PN96_RS12625 or PN96_RS12625_CDS_1
Lower bound,0.0
Upper bound,1000.0
